# Validation

OpenFisca-UK runs unit and integration tests on each new version (see [here](https://github.com/PSLmodels/openfisca-uk/tree/master/tests)).
In addition, the table below shows the aggregates produced by the model for the major taxes and benefits, and comparisons with UKMOD (latest [country report](https://www.iser.essex.ac.uk/research/publications/working-papers/cempa/cempa7-20.pdf)) and official sources.[^1]
UKMOD and administrative sources refer to 2018, and OpenFisca-UK is simulated on policy at the end of 2018.
Numbers are in billions of pounds.

[^1]: From the UKMOD country report: unless otherwise specified: Department for Work and Pensions https://www.gov.uk/government/publications/benefit-expenditure-and-caseload-tables-2018 ; Best Start Grant: https://www2.gov.scot/Topics/Statistics/Browse/Social-Welfare/SocialSecurityforScotland/BSGJune2019; Child tax credit and working tax credit: HMRC statistics 
https://www.gov.uk/government/statistics/child-and-working-tax-credits-statistics-finalised-annual-awards-2016-to-2017; Scottish Child Payment: Scottish Fiscal Commission https://www.fiscalcommission.scot/forecast/supplementary-costing-scottish-child-payment; Scottish Child Winter Heating Assistance: Scottish Fiscal Commission 
https://www.fiscalcommission.scot/forecast/supplementary-costing-child-winter-heating-assistance; Income tax: HMRC statistics https://www.gov.uk/government/statistics/income-tax-liabilities-statistics-tax-year-2014-to-2015-to-tax-year-2017-to-2018; National Insurance Contributions: ONS Blue Book Table 5.2.4s 

In [1]:
from openfisca_uk import Microsimulation
import pandas as pd

sim = Microsimulation()

VARIABLES = dict(
    income_tax="Income Tax",
    national_insurance="Non-employer National Insurance",
    employee_NI_class_1="Employee Class 1 NI",
    employer_NI_class_1="Employer Class 1 NI",
    NI_class_4="Class 2 & 4 NI",
    universal_credit="Universal Credit",
    tax_credits="Tax Credits",
    housing_benefit="Housing Benefit",
    state_pension="State Pension",
)


v = pd.DataFrame(columns=["ukmod", "external"])
v.loc["income_tax"] = 163.7, 190.0
v.loc["national_insurance"] = 64.0, 55.7
v.loc["employee_NI_class_1"] = 55.7, 52.0
v.loc["employer_NI_class_1"] = 77.8, 79.2
v.loc["NI_class_4"] = 5.0, 3.7
v.loc["universal_credit"] = 11.7, 8.1
v.loc["tax_credits"] = 13.9, 22.8
v.loc["housing_benefit"] = 15.1, 20.7
v.loc["state_pension"] = 86.4, 85.5

v["of"] = (
    sim.df(list(VARIABLES.keys()), map_to="household", period=2019).sum() / 1e9
)

v["of_ukmod"] = round(v.of / v.ukmod, 2)
v["of_external"] = round(v.of / v.external, 2)
v.of = v.of.round(1)
COLUMN_RENAMES = dict(
    of="OpenFisca-UK",
    ukmod="UKMOD",
    external="External",
    of_ukmod="OpenFisca-UK / UKMOD",
    of_external="OpenFisca-UK / External",
)
v[COLUMN_RENAMES.keys()].rename(index=VARIABLES, columns=COLUMN_RENAMES)

,OpenFisca-UK,UKMOD,External,OpenFisca-UK / UKMOD,OpenFisca-UK / External
Income Tax,170.1,163.7,190.0,1.04,0.90
Non-employer National Insurance,55.6,64.0,55.7,0.87,1.00
Employee Class 1 NI,51.7,55.7,52.0,0.93,0.99
Employer Class 1 NI,56.8,77.8,79.2,0.73,0.72
Class 2 & 4 NI,3.5,5.0,3.7,0.70,0.94
Universal Credit,13.5,11.7,8.1,1.15,1.66
Tax Credits,3.8,13.9,22.8,0.27,0.17
Housing Benefit,15.9,15.1,20.7,1.05,0.77
State Pension,103.0,86.4,85.5,1.19,1.20
